![](https://img-blog.csdnimg.cn/20200526084630342.png)

In [29]:
import cv2,os
import numpy as np
import pandas as pd
import find_color

 
#计算图片中每种颜色占比
def compute_colorArea(filename):
    color_areas = {}
    
    frame = cv2.imread(filename)
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    color_dict = find_color.getColorList()
    for d in color_dict:
        mask = cv2.inRange(hsv,color_dict[d][0],color_dict[d][1])
        #cv2.imwrite(d+'.jpg',mask)
        binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]
        binary = cv2.dilate(binary,None,iterations=2)
        
        cnts, hiera = cv2.findContours(binary.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        color_area = 0
        for c in cnts:
            color_area+=cv2.contourArea(c)
        color_areas[str(d)] = color_area
    
    return color_areas

#计算alpha中白色的面积，用于后续与背景图片中黑色部分面积相减
#得到背景中真实的各颜色占比
def alpha_whiteArea(filename):
    frame = cv2.imread(filename)
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    color_dict = find_color.getColorList()
    #for d in color_dict:
    mask = cv2.inRange(hsv,color_dict["white"][0],color_dict["white"][1])
    #cv2.imwrite(d+'.jpg',mask)
    binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]
    binary = cv2.dilate(binary,None,iterations=2)

    cnts, hiera = cv2.findContours(binary.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    color_area = 0
    for c in cnts:
        color_area+=cv2.contourArea(c)
    
    return color_area

if __name__ == '__main__':
    
    img_path = 'D:/A/Python/Datasets/CV/Segmentation/PersonSeg/background/'
    img_dirs = os.listdir(img_path)
    
    alpha_path = 'D:/A/Python/Datasets/CV/Segmentation/PersonSeg/png/'
    alpha_dirs = os.listdir(alpha_path)
        
    white_img_list = []
    for img_dir,alpha_dir in zip(img_dirs[:100],alpha_dirs[:100]):
        white_area = alpha_whiteArea(alpha_path+alpha_dir)
        color_areas = compute_colorArea(img_path+img_dir)
        
        #得到真实的黑色区域面积
        color_areas["black"] = color_areas["black"]-white_area
        #计算各颜色占比
        sum_area = sum(color_areas.values())
        color_ratios = {}
        for color in color_areas:
            ratio = color_areas[color]/sum_area
            if ratio<0.05:
                continue
            color_ratios[color] = ratio
        print(img_dir,'\t','颜色总数：',len(color_ratios),'\t',color_ratios)
        max_color = max(zip(color_ratios.values(), color_ratios.keys()))
        area = max_color[0]
        color = max_color[1]
        if (color == 'white' and len(color_ratios)<=2) or (color == 'gray' and len(color_ratios)<=2):
#         if len(color_ratios)<=2:
            white_img_list.append(img_dir)
            
#     # 保存文件
#     white_img_npy = np.array(white_img_list)
#     np.save('white_imgs.npy',white_img_npy)
#     '''
#     # 读取
#     a=np.load('white_imgs.npy')
#     a=a.tolist()
#     '''
    
#     white_img_csv = pd.DataFrame(white_img_list) 
#     white_img_csv.to_csv('white_imgs.csv',index=0)

train_10000.jpg.png 	 颜色总数： 2 	 {'gray': 0.1620909198170372, 'cyan': 0.7801675055358046}
train_10001.jpg.png 	 颜色总数： 4 	 {'gray': 0.14134132142290534, 'orange': 0.3750547881480127, 'yellow': 0.17837676498506266, 'green': 0.25579275425061915}
train_10002.jpg.png 	 颜色总数： 3 	 {'black': 0.4622408524060915, 'red2': 0.2652604887416088, 'orange': 0.24501519967393598}
train_10003.jpg.png 	 颜色总数： 2 	 {'black': 0.13807965502618041, 'gray': 0.7623615763812754}
train_10004.jpg.png 	 颜色总数： 7 	 {'black': 0.23243607547321649, 'gray': 0.12147738095931394, 'white': 0.09854094785711147, 'orange': 0.07450489165509425, 'yellow': 0.13716487834526048, 'green': 0.19872259479285842, 'blue': 0.052289010810401576}
train_10005.jpg.png 	 颜色总数： 2 	 {'gray': 0.3623237535000042, 'white': 0.6025274807447842}
train_10006.jpg.png 	 颜色总数： 4 	 {'black': 0.05576419883357812, 'gray': 0.08025388926121861, 'red2': 0.18714618773973676, 'orange': 0.5891624720158565}
train_10007.jpg.png 	 颜色总数： 5 	 {'black': 0.06986400969435842

In [49]:
if __name__ == '__main__':
    
    img_path = 'D:/A/Python/Datasets/CV/Segmentation/PersonSeg/background/'
    img_dirs = os.listdir(img_path)
    
    alpha_path = 'D:/A/Python/Datasets/CV/Segmentation/PersonSeg/png/'
    alpha_dirs = os.listdir(alpha_path)
        
    white_img_list = []
    for img_dir,alpha_dir in zip(img_dirs[:100],alpha_dirs[:100]):
        white_area = alpha_whiteArea(alpha_path+alpha_dir)
        color_areas = compute_colorArea(img_path+img_dir)
        
        #得到真实的黑色区域面积
        color_areas["black"] = color_areas["black"]-white_area
        #计算各颜色占比
        sum_area = sum(color_areas.values())
        color_ratios = {}
        for color in color_areas:
            ratio = color_areas[color]/sum_area
            color_ratios[color] = ratio
        color_ratios = {k:v for k, v in color_ratios.items() if v>=0.05}# 过滤掉ratio小于0.15的颜色
        sum_ratio = sum(color_ratios.values())
        color_ratios_new = {}
        for color in color_ratios:
            ratio = color_ratios[color]/sum_ratio
            color_ratios_new[color] = ratio
        print(img_dir,'\t','颜色总数：',len(color_ratios_new),'\t',color_ratios_new)
        max_color = max(zip(color_ratios_new.values(), color_ratios_new.keys()))
        area = max_color[0]
        color = max_color[1]
        cond1 = color_ratios_new[color]>0.5 and len(color_ratios_new)<=4 and color=='white'
        cond2 = color_ratios_new[color]>0.5 and len(color_ratios_new)<=4 and color=='gray'
        cond3 = color_ratios_new[color]>0.5 and len(color_ratios_new)<=4 and color=='red'
        cond4 = color_ratios_new[color]>0.5 and len(color_ratios_new)<=4 and color=='red2'
        cond5 = color_ratios_new[color]>0.5 and len(color_ratios_new)<=4 and color=='orange'
        cond6 = color_ratios_new[color]>0.5 and len(color_ratios_new)<=4 and color=='yellow'
        cond7 = color_ratios_new[color]>0.5 and len(color_ratios_new)<=4 and color=='green'
        cond8 = color_ratios_new[color]>0.5 and len(color_ratios_new)<=4 and color=='cyan'
        cond9 = color_ratios_new[color]>0.5 and len(color_ratios_new)<=4 and color=='blue'
        cond10 = color_ratios_new[color]>0.5 and len(color_ratios_new)<=4 and color=='purple'
        if (cond1) or (cond2) or (cond3) or (cond4) or (cond5) or (cond6) or (cond7) or (cond8) or (cond9) or (cond10):
        #         if (color_ratios_new[color]>0.5 and len(color_ratios_new)<=2) or (color_ratios_new[color]>0.5 and len(color_ratios_new)<=2):
#         if len(color_ratios)<=2:
            white_img_list.append(img_dir)


train_10000.jpg.png 	 颜色总数： 2 	 {'gray': 0.17202384765765294, 'cyan': 0.8279761523423471}
train_10001.jpg.png 	 颜色总数： 4 	 {'gray': 0.14869180742454804, 'orange': 0.3945595935536615, 'yellow': 0.18765328724225822, 'green': 0.26909531177953233}
train_10002.jpg.png 	 颜色总数： 3 	 {'black': 0.4753038462621567, 'red2': 0.2727567888125604, 'orange': 0.25193936492528285}
train_10003.jpg.png 	 颜色总数： 2 	 {'black': 0.1533466596263609, 'gray': 0.846653340373639}
train_10004.jpg.png 	 颜色总数： 7 	 {'black': 0.2539908072442849, 'gray': 0.13274246688669886, 'white': 0.10767904612865808, 'orange': 0.08141402979980157, 'yellow': 0.1498847289756933, 'green': 0.21715093995782556, 'blue': 0.05713798100703776}
train_10005.jpg.png 	 颜色总数： 2 	 {'gray': 0.37552292067450527, 'white': 0.6244770793254948}
train_10006.jpg.png 	 颜色总数： 4 	 {'black': 0.061123055928118786, 'gray': 0.08796616941278064, 'red2': 0.20513065979999784, 'orange': 0.6457801148591027}
train_10007.jpg.png 	 颜色总数： 5 	 {'black': 0.07730375034372264, 

['train_10000.jpg.png',
 'train_10003.jpg.png',
 'train_10005.jpg.png',
 'train_10006.jpg.png',
 'train_10009.jpg.png',
 'train_10010.jpg.png',
 'train_10012.jpg.png',
 'train_10013.jpg.png',
 'train_10018.jpg.png',
 'train_10020.jpg.png',
 'train_10021.jpg.png',
 'train_10023.jpg.png',
 'train_10025.jpg.png',
 'train_10027.jpg.png',
 'train_10029.jpg.png',
 'train_10031.jpg.png',
 'train_10032.jpg.png',
 'train_10038.jpg.png',
 'train_10041.jpg.png',
 'train_10042.jpg.png',
 'train_10045.jpg.png',
 'train_10047.jpg.png',
 'train_10048.jpg.png',
 'train_10051.jpg.png',
 'train_10053.jpg.png',
 'train_10054.jpg.png',
 'train_10055.jpg.png',
 'train_10057.jpg.png',
 'train_10058.jpg.png',
 'train_10059.jpg.png',
 'train_10061.jpg.png',
 'train_10062.jpg.png',
 'train_10063.jpg.png',
 'train_10064.jpg.png',
 'train_10066.jpg.png',
 'train_10069.jpg.png',
 'train_10071.jpg.png',
 'train_10072.jpg.png',
 'train_10075.jpg.png',
 'train_10076.jpg.png',
 'train_10077.jpg.png',
 'train_10082.jp